In [ ]:

from requests import session
import matplotlib.pyplot as plt
import pandas as pd
import json
import copy
import numpy as np, scipy.stats as st
import json
from tqdm import tqdm
import pickle
import logging
import  argparse
import math
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from itertools import chain
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
logging.basicConfig(level=logging.INFO)
from sklearn import preprocessing
import matplotlib
from re import S
from scipy.stats.stats import pearsonr
from sklearn.calibration import calibration_curve
import matplotlib
from sklearn.metrics import roc_auc_score
import random
import sys
import os
# sys import ..
sys.path.append("../")
from data_split import *
from metrics_get import *




In [ ]:
path = 'featureframe_user_study.pkl'
data = pickle.load(open(path, 'rb'))
splitbyusers = True
output_path = '.'
test_percentage = 0.001
val_percentage = 0.001
REMOVE_S_AND_R = True # remove shown and replay
feature_dict = {'Measurements: compCharLen, confidence, documentLength, numLines, numTokens, promptCharLen, promptEndPos, quantile': 0,
'edit percentage': 1, 'time_in_state': 2, 'session_features':3, 'suggestion_label':4, 'prompt_label':5,
'suggestion_embedding':6, 'prompt_embedding':7, 'suggestion_text_features':8, 'prompt_text_features':9, 'user_id':10, 'statename':11, 'labeled_state': 12}
# split into train and test
SEQUENCE_MODE = False # keep session as a sequence or split it into events
SPLIT_BY_USER = bool(splitbyusers) # otherwise split by session uniformly
ADD_PREVIOUS_STATES = True
PREDICT_ACTION = True # Otherwise predict time in state
NORMALIZE_DATA = False # normalize data

previous_states_to_keep = 5
if not PREDICT_ACTION and SPLIT_BY_USER:
    raise ValueError('Cannot predict time and split by user')




In [ ]:
MAX_TRIALS = 21 # number of programmers
so_far_count = 0
aucs_with = []
aucs_without = []
accs_with = []
accs_without = []
test_set_seenns = []
while so_far_count < MAX_TRIALS:
    random_number = np.random.randint(0, 100000)

    # without latent state
    features_to_keep = np.array([0,3,4,5,8,9, 10])
    label_index = np.array([11])
    df_observations_features, df_observations_labels = get_features_labels_user_study(path, features_to_keep, label_index, REMOVE_S_AND_R)

    X_train, X_test, X_val, y_train, y_test, y_val = process_data_user_study(df_observations_features, df_observations_labels,
    REMOVE_S_AND_R, SEQUENCE_MODE, SPLIT_BY_USER, ADD_PREVIOUS_STATES, PREDICT_ACTION, NORMALIZE_DATA,
    test_percentage, val_percentage, previous_states_to_keep, random_number)
    # check if X_test array is already been seen
    for seen in test_set_seenns:
        if np.array_equal(X_test, seen):
            continue
        
    test_set_seenns.append(X_test)
    so_far_count += 1
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    y_pred_proba = model.predict_proba(X_test)
    y_pred_proba = y_pred_proba[:,1]
    auc = roc_auc_score(y_test, y_pred_proba)
    aucs_without.append(auc)
    accs_without.append(accuracy)
    # with latent state
    features_to_keep = np.array([0,3,4,5,8,9, 10,12])
    label_index = np.array([11])
    df_observations_features, df_observations_labels = get_features_labels_user_study(path, features_to_keep, label_index, REMOVE_S_AND_R)

    X_train, X_test, X_val, y_train, y_test, y_val = process_data_user_study(df_observations_features, df_observations_labels,
    REMOVE_S_AND_R, SEQUENCE_MODE, SPLIT_BY_USER, ADD_PREVIOUS_STATES, PREDICT_ACTION, NORMALIZE_DATA,
    test_percentage, val_percentage, previous_states_to_keep, random_number)
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    y_pred_proba = model.predict_proba(X_test)
    y_pred_proba = y_pred_proba[:,1]
    auc = roc_auc_score(y_test, y_pred_proba)
    aucs_with.append(auc)
    accs_with.append(accuracy)
    print(f'aucs_with: {aucs_with}')
    print(f'aucs_without: {aucs_without}')
    print(f'accs_with: {accs_with}')
    print(f'accs_without: {accs_without}')

In [ ]:
# print mean and standard error
print(f'WITH latent state model: accuracy = {np.mean(accs_with)} +- {np.std(accs_with)/np.sqrt(len(accs_with))}')
print(f'WITHOUT latent state model: accuracy = {np.mean(accs_without)} +- {np.std(accs_without)/np.sqrt(len(accs_without))}')
print(f'WITH latent state model auc = {np.mean(aucs_with)} +- {np.std(aucs_with)/np.sqrt(len(aucs_with))}')
print(f'WITHOUT latent state model auc = {np.mean(aucs_without)} +- {np.std(aucs_without)/np.sqrt(len(aucs_without))}')

In [ ]:
# perform paired t-test on accuracy and aucs for with and without
from scipy.stats import ttest_rel

test_auc = ttest_rel(aucs_with, aucs_without)
test_acc = ttest_rel(accs_with, accs_without)
print("T-test auc")
print(test_auc)
print("T-test acc")
print(test_acc)
